In [1]:
import torch
import timm

model = torch.load('TOP_UWF_ema_model.pt')
# you can use the jit version for better compatability
# model = torch.jit.load('TOP_UWF_ema_model_jit.pt')
model.eval().cpu()

print(f'timm version: {timm.__version__}')
# hacky fix in case you are using timm version > 4.10; please use the versions in the requirements.txt and/or try to jit version if these tests don't work
model.global_pool.flatten = torch.nn.Flatten(1)

#### TEST INPUT-OUTPUT
# b, c, h, w
test_input = torch.zeros(1, 2, 384, 512)
# expected output for all zero tensor of shape (1,2,384,512), rounded to four decimal places; in logit space
expected_output = torch.tensor([-2.7877, -2.8305, -2.9748, -2.2224, -3.2804, -2.4505,  1.1078,  1.3486])
with torch.no_grad():
    actual_output = model(test_input).flatten()
    print(f'Actual output:   {actual_output}')
    print(f'Expected output: {expected_output}')
    print(f'Diff (rounded):  {expected_output - actual_output.round(decimals=4)}')

timm version: 0.5.4
Actual output:   tensor([-2.7877, -2.8305, -2.9748, -2.2224, -3.2804, -2.4505,  1.1078,  1.3486])
Expected output: tensor([-2.7877, -2.8305, -2.9748, -2.2224, -3.2804, -2.4505,  1.1078,  1.3486])
Diff (rounded):  tensor([0., 0., 0., 0., 0., 0., 0., 0.])


In [3]:
#### TEST IMAGE
from torchvision import transforms as T
from PIL import Image

targets = ['MH', 'RP', 'AMD', 'RVO', 'RD', 'Gla', 'DR', 'any_retina_disease']

# Exact means and stds from the training set, third dimension added for compatibility with plotting functions
norm_means = [0.22578795, 0.23797078, 1]
norm_stds = [0.14651306, 0.11282759, 1]
resolution = (384, 512)
norm_transform = T.Compose([
    T.ToTensor(),
    T.Resize(resolution),                                  
    T.Normalize(norm_means, norm_stds),
    # remove third channel if present
    T.Lambda(lambda x: x[:2,...])
])

# test image from the external validation set in our study, originally used by Antaki et al. in their external validation set
# image shows RP
img_url = 'https://i.redd.it/hij0f9pkqn441.jpg'
!wget https://i.redd.it/hij0f9pkqn441.jpg?raw=true -O test_img_RP.jpg
img = Image.open('test_img_RP.jpg')
# transform and add a dummy batch dim by unsqueezing
img_normalized = norm_transform(img).unsqueeze(0)
# expected output (in probability rather than logit space)
expected_output = torch.tensor([0.0622, 0.9177, 0.0537, 0.0322, 0.0666, 0.0957, 0.0739, 0.9620])

with torch.no_grad():
    actual_output = model(img_normalized).flatten()
    # apply sigmoid to convert to probs
    actual_output = torch.sigmoid(actual_output)
    print(f'Actual output:   {actual_output}')
    print(f'Expected output: {expected_output}')
    print(f'Diff (rounded):  {expected_output - actual_output.round(decimals=4)}')

print('\nThis external validation set image (adapted from Antaki et al.\'s external validation set, which also includes this image) shows RP.\n'\
      'So, our model should predict RP and "any" (i.e. any disease) with high probabilities,\n'\
      'and the rest with low probabilities (bearing in mind that we used label smoothing,\n'\
      'so our model tries to predict approximately 5% if a label is absent and 99% if it is present.)'\
      '\n\nHere is what we got:')

for predicted_probability, label_name in zip(actual_output, targets):
    print(f'Predicted {label_name[:3]:>3} with probability: {predicted_probability.item():.4f} (or {predicted_probability.item()*100:.2f}%)')
    
print('\nIf all worked well, RP and any should be predicted with ps of 91.77% and 96.20%, respectively, whereas the other labels are roughly around 5%. Neat!')

--2022-07-24 18:51:04--  https://i.redd.it/hij0f9pkqn441.jpg?raw=true
Resolving i.redd.it (i.redd.it)... 199.232.57.140, 2a04:4e42:4b::396
Connecting to i.redd.it (i.redd.it)|199.232.57.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45020 (44K) [image/jpeg]
Saving to: ‘test_img_RP.jpg’

test_img_RP.jpg     100%[===================>]  43.96K  --.-KB/s    in 0.01s   

2022-07-24 18:51:04 (3.63 MB/s) - ‘test_img_RP.jpg’ saved [45020/45020]

Actual output:   tensor([0.0622, 0.9177, 0.0537, 0.0322, 0.0666, 0.0957, 0.0739, 0.9620])
Expected output: tensor([0.0622, 0.9177, 0.0537, 0.0322, 0.0666, 0.0957, 0.0739, 0.9620])
Diff (rounded):  tensor([0., 0., 0., 0., 0., 0., 0., 0.])

This external validation set image (adapted from Antaki et al.'s external validation set, which also includes this image) shows RP.
So, our model should predict RP and "any" (i.e. any disease) with high probabilities,
and the rest with low probabilities (bearing in mind that we used labe